In [1]:
from src.scripts.preprocess import *
from src.scripts.minimize_lkh import Model
from src.scripts.utils import display_return_stats, find_case, transform_params

In [2]:
def main(gamma, delta, sigma, k, a, b, pi, impose_alpha=False, stockidx=1, nopi=0, dok=1, bankhand=2):
    logmk = load_index_data()
    logrf = load_tbills_data()
    x = load_venture_data(0, 0, 0)

    print(f"number of observations: {x.shape[0]}")
    x["ddate"] = to_decimal_date(x["round_date"])
    x = x.sort_values(by=["ddate", "company_num"]).drop(columns=["ddate"])

    display_return_stats(x)
    
    c = sum((x["exit_type"]==3) & (x["exit_date"] != -99)) / sum((x["exit_type"] == 3))
    print(f'\tPercent of bankrupt have good data. Using this parameter in simulation: {c*100:.2f}%')
    
    good_exit = x["exit_type"].isin([1,2,5,6])
    good_date = x["exit_date"] != -99
    good_return = x["return_usd"] > 0
    d = (good_exit & good_date & good_return).sum() / good_exit.sum()
    print(f'\tPercent of valuations (ipo, acquired, new round) that have good data: {d*100:.2f}%\n\n\n\n')
    
    minage = 0.25
    logv = np.arange(-7, 7.1, 0.1)
    pi = 0 if nopi == 1 else pi
    xc = find_case(x, dok, bankhand)
    mask = [impose_alpha!=1, stockidx>0, True, dok!=0, True, True, nopi!=1]
    tpar0 = transform_params(gamma, delta, sigma, k, a, b, pi, mask)
    
    model = Model(x, xc, logrf, logmk, minage, c, d, logv, mask, stockidx, dok, start_year=1987, sample_size=54)
    model.model_likelyhood(tpar0)
    return model.optimize_likelyhood(tpar0, mask, maxiter=30)

In [3]:
gamma0 = 0.01
delta0 = 1.5
sigma0 = 0.9
k0 = 0.1
a0 = 1
b0 = 3
pi0 = 0.01

res = main(gamma0, delta0, sigma0, k0, a0, b0, pi0)

number of observations: 16642
Note: following refers to round, not company.
Round may end in another round, though company eventually goes public
	Percent bankrupt: 9.19%
	Percent ipo: 21.20%
	Percent acquired: 20.41%
	Percent with subsequent round: 0.00%
	Percent Private: 45.49%
	Percent Ipo registered: 3.71%
	Percent fate unknown: 0.00%
	Percent of bankrupt have good data. Using this parameter in simulation: 95.09%
	Percent of valuations (ipo, acquired, new round) that have good data: 47.66%




iter gamma     delta     sigma     k         a         b         pim       lkh       
0    0.0829    1.4955    -0.4433   -2.3743   0.1808    3.0904    -4.5950   55439.1755
1    0.0327    1.4949    -0.4992   -2.3855   0.2121    3.1076    -4.5949   54708.5503
2    0.0233    1.4851    -0.5451   -2.3929   0.1499    3.0995    -4.5935   54614.4485
3    0.0226    1.1854    -0.5221   -2.4324   0.0354    3.4097    -4.5507   54470.7840
4    0.0198    0.3068    -0.6098   -1.9296   -0.0113   3.5970    -4

In [4]:
res

,gamma,delta,sigma,k,a,b,pim
value,0.008594,-2.175729,0.466514,0.206822,0.690855,5.227605,0.063181
std,0.001806,0.038723,0.011514,0.026274,0.016718,0.084857,0.145633
